In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import numpy.random
import tensorflow as tf
import datetime
import struct
import time
from sklearn import linear_model
from sklearn import svm
import sklearn.model_selection
import sklearn.metrics
import sklearn.preprocessing

root_dir = "D:/Jupyter/";
logs_dir = root_dir + "Logs/"
data_dir = root_dir + 'Datasets/'

def mnist_read_imgs(fname):
    with open(fname, mode='rb') as f:
        (_, img_num, img_xsize, img_ysize) = struct.unpack('>IIII',f.read(4 * 4))
        data_img = np.fromfile(f, dtype=np.uint8).reshape(img_num, img_xsize, img_ysize)
    return data_img

def mnist_read_lbls(fname):
    with open(data_dir + 'MNIST/train-labels.idx1-ubyte', mode='rb') as f:
        (_, lab_num) = struct.unpack('>II', f.read(4 * 2))
        data_lab = np.fromfile(f, dtype=np.uint8)
    return data_lab

def minibatch(X, y, num=1000):
    inds = np.random.choice(range(X.shape[0]), size=num)
    return X[inds], y[inds]

In [4]:
src_X = mnist_read_imgs(data_dir+'MNIST/train-images.idx3-ubyte')
src_y = mnist_read_lbls(data_dir+'MNIST/train-labels.idx1-ubyte')

random_seed = 42
(dev_X, test_X, dev_y, test_y) = sklearn.model_selection.train_test_split(src_X, src_y, random_state=random_seed, test_size=0.2)
(train_X, valid_X, train_y, valid_y) = sklearn.model_selection.train_test_split(dev_X, dev_y, random_state=random_seed, test_size=0.2)

## 1D Neural Networks
### Prepare data

In [5]:
def mnist1d_transform_imgs(x):
    return x.reshape(x.shape[0], x.shape[1] * x.shape[2]) / 255

def mnist1d_transform_lbls(y):
    return np.array([1.0*(y==i) for i in range(10)]).transpose()

In [6]:
(train1d_X, valid1d_X, test1d_X) = (mnist1d_transform_imgs(x) for x in (train_X, valid_X, test_X))
(train1d_y, valid1d_y, test1d_y) = (mnist1d_transform_lbls(y) for y in (train_y, valid_y, test_y))

### 0-hidden layer network
Current accuracy on validation is __92.5%__

In [13]:
tf.reset_default_graph()
dt_now = datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S")
log_dir = root_dir + 'Logs/' + dt_now + '-LR-S-B5k'

tf_LearningRate = tf.placeholder(shape=(), name='LearningRate', dtype=tf.float32)
tf_Input = tf.placeholder(shape=(None, 784), name='Input', dtype=tf.float32)
tf_Labels = tf.placeholder(shape=(None, 10), name='Labels', dtype=tf.float32)

tf_Output = tf.layers.dense(tf_Input, 10, use_bias=True, name='LogisticRegression')

tf_OutProb = tf.nn.softmax(tf_Output)

tf_Error = -tf.reduce_mean(tf.reduce_sum(tf_Labels * tf.log(tf_OutProb), reduction_indices=1))
tf_Optimizer = tf.train.GradientDescentOptimizer(tf_LearningRate)
tf_TrainStep = tf_Optimizer.minimize(tf_Error)

tf_Initialize = tf.global_variables_initializer()

tf_ErrorSummary = tf.summary.scalar('Error', tf_Error)
tf_FW = tf.summary.FileWriter(log_dir, tf.get_default_graph())

In [14]:
num_epochs = 100
num_steps = 100
batch_size = 5000
learning_rate = 0.5
fulltrain_batch = {tf_Input: train1d_X, tf_Labels:train1d_y}
validation_batch = {tf_Input: valid1d_X, tf_Labels: valid1d_y}
test_batch = {tf_Input: test1d_X, tf_Labels: test1d_y}
with tf.Session() as sess:
    tf_Initialize.run()
    for i in range(num_epochs):
        if i > 10:
            learning_rate = 0.2
        if i > 50:
            learning_rate = 0.1
        tX, ty = minibatch(train1d_X, train1d_y, num=batch_size)
        batch = {tf_Input: tX, tf_Labels:ty, tf_LearningRate: learning_rate}
        for j in range(num_steps):
            tf_TrainStep.run(feed_dict=batch)
        
        sumstr = tf_ErrorSummary.eval(feed_dict=validation_batch)
        tf_FW.add_summary(sumstr, i)
    train1d_nn0_prob = tf_OutProb.eval(feed_dict=fulltrain_batch)
    valid1d_nn0_prob = tf_OutProb.eval(feed_dict=validation_batch)
    test1d_nn0_prob = tf_OutProb.eval(feed_dict=test_batch)

In [99]:
display(sklearn.metrics.confusion_matrix(valid_y, np.argmax(valid1d_nn0_prob, axis=1)))
sklearn.metrics.accuracy_score(valid_y, np.argmax(valid1d_nn0_prob, axis=1))

array([[ 933,    0,    2,    1,    5,   11,    2,    0,    8,    1],
       [   0, 1076,    2,    0,    1,    6,    1,    1,    9,    3],
       [   8,   12,  818,   18,   10,    4,   12,   13,   20,    8],
       [   2,    2,   23,  928,    2,   25,    2,    9,   26,    3],
       [   1,    4,    7,    1,  887,    0,    8,    3,   11,   39],
       [   5,    6,    7,   29,    5,  753,   15,    1,   19,    4],
       [   4,    5,    5,    0,   11,   16,  903,    0,    4,    0],
       [   1,    7,    7,    3,    5,    3,    1,  901,    0,   50],
       [   4,   16,    7,   21,    1,   23,    5,    2,  816,   27],
       [   5,    4,    0,    8,   26,    6,    0,   25,    3,  863]], dtype=int64)

0.92479166666666668

### 1-hidden layer network
Current accuracy on validation is __97.3%__

In [15]:
tf.reset_default_graph()
dt_now = datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S")
log_dir = root_dir + 'Logs/' + dt_now + '-H200-LR-S-B5k'

tf_LearningRate = tf.placeholder(shape=(), name='LearningRate', dtype=tf.float32)
tf_Input = tf.placeholder(shape=(None, 784), name='Input', dtype=tf.float32)
tf_Labels = tf.placeholder(shape=(None, 10), name='Labels', dtype=tf.float32)

tf_Hidden = tf.layers.dense(tf_Input, 200, use_bias=True, activation=tf.nn.elu, name='Hidden-1')
tf_Output = tf.layers.dense(tf_Hidden, 10, use_bias=True, name='SoftMax')

tf_OutProb = tf.nn.softmax(tf_Output)

tf_Error = -tf.reduce_mean(tf.reduce_sum(tf_Labels * tf.log(tf_OutProb), reduction_indices=1))
tf_Optimizer = tf.train.GradientDescentOptimizer(tf_LearningRate)
tf_TrainStep = tf_Optimizer.minimize(tf_Error)

tf_Initialize = tf.global_variables_initializer()

tf_ErrorSummary = tf.summary.scalar('Error', tf_Error)
tf_FW = tf.summary.FileWriter(log_dir, tf.get_default_graph())

In [16]:
num_epochs = 100
num_steps = 100
batch_size = 5000
learning_rate = 0.5
fulltrain_batch = {tf_Input: train1d_X, tf_Labels:train1d_y}
validation_batch = {tf_Input: valid1d_X, tf_Labels: valid1d_y}
test_batch = {tf_Input: test1d_X, tf_Labels: test1d_y}
with tf.Session() as sess:
    tf_Initialize.run()
    for i in range(num_epochs):
        if i > 10:
            learning_rate = 0.2
        if i > 50:
            learning_rate = 0.1
        tX, ty = minibatch(train1d_X, train1d_y, num=batch_size)
        batch = {tf_Input: tX, tf_Labels:ty, tf_LearningRate: learning_rate}
        for j in range(num_steps):
            tf_TrainStep.run(feed_dict=batch)
        
        sumstr = tf_ErrorSummary.eval(feed_dict=validation_batch)
        tf_FW.add_summary(sumstr, i)
    train1d_nn1_prob = tf_OutProb.eval(feed_dict=fulltrain_batch)
    valid1d_nn1_prob = tf_OutProb.eval(feed_dict=validation_batch)
    test1d_nn1_prob = tf_OutProb.eval(feed_dict=test_batch)

In [96]:
sklearn.metrics.accuracy_score(valid_y, np.argmax(valid1d_nn1_prob, axis=1))

0.97281249999999997

### 2-hidden layer network
With 300-300 combination, dropout and res-net hack arrived at __98.1%__

In [17]:
tf.reset_default_graph()
dt_now = datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S")
log_dir = root_dir + 'Logs/' + dt_now + '-H300-H300-DROP-RES-LR-S-B5k-GD'

tf_Training = tf.placeholder(shape=(), name='Training', dtype=tf.bool)
tf_LearningRate = tf.placeholder(shape=(), name='LearningRate', dtype=tf.float32)
tf_Input = tf.placeholder(shape=(None, 784), name='Input', dtype=tf.float32)
tf_Labels = tf.placeholder(shape=(None, 10), name='Labels', dtype=tf.float32)

tf_Hidden1 = tf.layers.dense(tf_Input, 300, use_bias=True, activation=tf.nn.relu, name='Hidden-1')
tf_Hidden2 = tf.layers.dense(tf.layers.dropout(tf_Hidden1, training=tf_Training),
                             300, use_bias=True, activation=tf.nn.relu, name='Hidden-2')
tf_Output = tf.layers.dense(tf.layers.dropout(tf.concat([tf_Hidden1, tf_Hidden2], axis=1), training=tf_Training),
                            10, use_bias=True, name='SoftMax')

tf_OutProb = tf.nn.softmax(tf_Output)

#tf_Error = -tf.reduce_mean(tf.reduce_sum(tf_Labels * tf.log(tf_OutProb), reduction_indices=1))
tf_Error = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_Labels, logits=tf_Output))
tf_TrainStep = tf.train.GradientDescentOptimizer(tf_LearningRate).minimize(tf_Error)

tf_Initialize = tf.global_variables_initializer()

tf_ErrorSummary = tf.summary.scalar('Error', tf_Error)
tf_FW = tf.summary.FileWriter(log_dir, tf.get_default_graph())

In [19]:
num_epochs = 100
num_steps = 100
batch_size = 5000
learning_rate = 0.5
fulltrain_batch = {tf_Training: False, tf_Input: train1d_X, tf_Labels:train1d_y}
validation_batch = {tf_Training: False, tf_Input: valid1d_X, tf_Labels: valid1d_y}
test_batch = {tf_Training: False, tf_Input: test1d_X, tf_Labels: test1d_y}
with tf.Session() as sess:
    tf_Initialize.run()
    for i in range(num_epochs):
        if i > 10:
            learning_rate = 0.2
        if i > 25:
            learning_rate = 0.1
        tX, ty = minibatch(train1d_X, train1d_y, num=batch_size)
        batch = {tf_Training: True, tf_Input: tX, tf_Labels:ty, tf_LearningRate: learning_rate}
        for j in range(num_steps):
            tf_TrainStep.run(feed_dict=batch)
        
        sumstr = tf_ErrorSummary.eval(feed_dict=validation_batch)
        tf_FW.add_summary(sumstr, i)
    train1d_nn2_prob = tf_OutProb.eval(feed_dict=fulltrain_batch)
    valid1d_nn2_prob = tf_OutProb.eval(feed_dict=validation_batch)
    test1d_nn2_prob = tf_OutProb.eval(feed_dict=test_batch)

In [144]:
sklearn.metrics.accuracy_score(valid_y, np.argmax(valid1d_nn2_prob, axis=1))

0.98052083333333329

In [20]:
display(sklearn.metrics.accuracy_score(train_y, np.argmax(train1d_nn2_prob, axis=1)))
display(sklearn.metrics.accuracy_score(valid_y, np.argmax(valid1d_nn2_prob, axis=1)))
display(sklearn.metrics.accuracy_score(test_y, np.argmax(test1d_nn2_prob, axis=1)))

0.99843749999999998

0.98062499999999997

0.98024999999999995

In [21]:
display(sklearn.metrics.accuracy_score(train_y, np.argmax(train1d_nn1_prob, axis=1)))
display(sklearn.metrics.accuracy_score(valid_y, np.argmax(valid1d_nn1_prob, axis=1)))
display(sklearn.metrics.accuracy_score(test_y, np.argmax(test1d_nn1_prob, axis=1)))

0.99382812499999995

0.97437499999999999

0.97233333333333338

In [23]:
display(sklearn.metrics.accuracy_score(train_y, np.argmax(train1d_nn0_prob, axis=1)))
display(sklearn.metrics.accuracy_score(valid_y, np.argmax(valid1d_nn0_prob, axis=1)))
display(sklearn.metrics.accuracy_score(test_y, np.argmax(test1d_nn0_prob, axis=1)))

0.92942708333333335

0.92343750000000002

0.92133333333333334